### Creating the Noisy XNLI test set

In [ ]:
import pandas as pd
language = 'hi'
from datasets import load_dataset
ds_orig = load_dataset('xnli', language,split='test')
df_test = pd.DataFrame(ds_orig)

In [ ]:
import pickle

with open('/Noise Dics/wiki_typos_hi_p320000_l3277.pkl', 'rb') as f:
    ds_t = pickle.load(f)

max number of changes in a row : max_aug <br>
replcace p percent of the tokens in a row :  percent_to_replace <br>



In [ ]:
df = df_test.copy()
# ds_t = wten.output
# Function to replace a percentage of tokens in a sentence
def replace_noisy(sentence_list, percent_to_replace=30 , max_aug=4):

    sentence = sentence_list[0]+" * "+sentence_list[1]
#     print(sentence)
    tokens = nltk.word_tokenize(sentence)
    num_tokens_to_replace = int((len(tokens)-1) * (percent_to_replace / 100))
    index_list = random.sample(range(0, len(tokens) ),len(tokens)-1)
    cc =0
    for i in index_list:
        if (cc == num_tokens_to_replace) or (cc== max_aug):
            break
        index_to_replace = i
        original_token = tokens[index_to_replace]
        random.shuffle(ds_t)
        for original, replacement in ds_t:
            if original_token.startswith(original):
                tokens[index_to_replace] = original_token.replace(original, replacement.lower())
                cc = cc +1
                break
    
    res=' '.join(tokens)
    return res.split('*')[0][:-1],res.split('*')[1][1:]

# Applying the custom function to the 'sentence' column with a specified percentage (e.g., 30%)
df['combined'] = df[['premise','hypothesis']].apply(lambda x: replace_noisy(x, percent_to_replace=20,max_aug = 4),axis=1)
df[['premise', 'hypothesis']] = pd.DataFrame(df['combined'].tolist(), index=df.index)

#Dropping the temporary 'combined' column
df = df.drop(columns=['combined'])

In [ ]:
df

In [ ]:
from datasets import Dataset
ds_noisy = Dataset.from_pandas(df , features = ds_orig.features,preserve_index=False)
ds_noisy.save_to_disk("xnli_WikiTypos/"+language+"_noisy.hf")

In [ ]:
pd.set_option('display.max_colwidth', 150)
df[['premise','augmented']].head(20)